In [ ]:
# ===== Модули Обновлено =====

from grid.model.navigation.objectinspection import ObjectInspect
from grid.model.navigation.visualservoing import VisualServoing
from grid.model.perception.detection.rt_detr import RT_DETR

import airgen
import numpy as np
import rerun as rr
import cv2
import time
from math import cos, sin, pi

# Инициализация моделей
nav_inspect = ObjectInspect()
nav_visualservoing = VisualServoing()
detection_rt_detr = RT_DETR(use_local=True)  # Инициализация модели детекции

In [ ]:
# ===== Подключение к дрону и взлёт =====

client = airgen.MultirotorClient()
client.confirmConnection()
client.enableApiControl(True)
client.armDisarm(True)
# ===== Взлет дрона =====
#client.takeoffAsync().join()

# ===== Посадка дрона =====
# client.landAsync().join()

Connected!
Client Ver:1 (Min Req: 1), Server Ver:1 (Min Req: 1)



True

In [ ]:
# ===== Создание клиента сцены =====

scene_client = airgen.VehicleClient()
scene_objects = scene_client.simListSceneObjects()
cameras = [obj for obj in scene_objects if 'camera' in obj.lower()]

print("Available cameras:")
for camera in cameras:
    print(camera)

Available cameras:
PlayerCameraManager_2147482445
CameraManagerActor
ExternalCamera
BP_PIPCamera_C_2147482405
BP_PIPCamera_C_2147482393
BP_PIPCamera_C_2147482381
BP_PIPCamera_C_2147482369
BP_PIPCamera_C_2147482357


In [ ]:
# ===== Получение данных LiDAR + визуализация =====

lidar_data = client.getLidarData()
points = np.array(lidar_data.point_cloud, dtype=np.float32).reshape(-1, 3)
rr.log('lidar/points', rr.Points3D(points))

In [ ]:
# ===== Получение и логирование изображения с камеры =====

image_data, _ = client.getImages("front_center", [airgen.ImageType.Scene])[0]
rgb_image = cv2.cvtColor(image_data, cv2.COLOR_RGB2BGR)

In [ ]:
# ===== Получение и логирование GPS =====

gps_data = client.getGpsData()
geo = gps_data.gnss.geo_point
gps_position = np.array([[geo.latitude, geo.longitude, geo.altitude]], dtype=np.float32)
rr.log("drone/gps", rr.Points3D(gps_position))

In [ ]:
from grid.model.perception.depth.metric3d import Metric3D

# ===== Инициализация =====
depth_model = Metric3D()
depth_map = depth_model.run(rgb_image)

result = detection_rt_detr.run(input=rgb_image.copy(), confidence_threshold=0.3)

if isinstance(result, dict) and 'boxes' in result:
    filtered_boxes = []
    filtered_labels = []
    filtered_distances = []

    ignore_labels = ["grass", "floor", "ground", "road", "terrain"]
    min_distance = 2
    max_distance = 20

    # Центр изображения — точка откуда "исходит БПЛА"
    img_center = (rgb_image.shape[1] // 2, rgb_image.shape[0] // 2)

    image_with_lines = rgb_image.copy()

    for box, label in zip(result["boxes"], result["labels"]):
        x, y, w, h = map(int, box)
        cx = x + w // 2
        cy = y + h // 2

        if 0 <= cy < depth_map.shape[0] and 0 <= cx < depth_map.shape[1]:
            distance = depth_map[cy, cx]

            if (min_distance <= distance <= max_distance) and (label.lower() not in ignore_labels):
                filtered_boxes.append([x, y, w, h])
                filtered_labels.append(label)
                filtered_distances.append(distance)

                # ===== Отрисовка линии до объекта =====
                line_color = (0, 255, 0) if distance > 10 else (0, 255, 255) if distance > 5 else (0, 0, 255)
                cv2.line(image_with_lines, img_center, (cx, cy), line_color, 2)

                # Подпись расстояния у центра объекта
                cv2.putText(image_with_lines, f"{distance:.1f}m", (cx + 5, cy),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, line_color, 2)

    # ===== Отрисовка боксов =====
    for i, box in enumerate(filtered_boxes):
        x, y, w, h = box
        cv2.rectangle(image_with_lines, (x, y), (x + w, y + h), (255, 0, 0), 2)
        cv2.putText(image_with_lines, filtered_labels[i], (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        rr.log(
            f"drone/object_{i}",
            rr.Boxes2D(np.array([[x, y, w, h]]), labels=[f"{filtered_labels[i]} - {filtered_distances[i]:.1f}m"])
        )

    # ===== Лог финального изображения =====
    rr.log("drone/camera", rr.Image(image_with_lines))

else:
    print("❗ Объекты не обнаружены.")


❗ Объекты не обнаружены.


In [ ]:
# ==== Движение БПЛА: шаблон ====

# Повернуть на определённый угол (по оси Yaw, в градусах)
# Положительное значение — поворот по часовой, отрицательное — против часовой
#yaw_angle = 45  # <- Измени это значение для другого угла поворота
#client.rotateByYawRateAsync(yaw_rate=yaw_angle, duration=2).join()

# Движение вперёд
#forward_distance = 3  # <- Измени это значение, чтобы задать дальность движения вперёд
#client.moveByVelocityAsync(vx=1.0, vy=0.0, vz=0.0, duration=forward_distance).join()

# Движение влево (vy > 0) или вправо (vy < 0)
#side_distance = 3  # <- Измени это значение для расстояния вбок
#client.moveByVelocityAsync(vx=0.0, vy=1.0, vz=0.0, duration=side_distance).join()

# Примечание:
# - vx — движение вперёд/назад (м/с), vy — влево/вправо, vz — вверх/вниз
# - duration — время движения в секундах
# - можно комбинировать движения, задавая одновременно vx, vy, vz


In [ ]:
#!pip install scikit-learn

from sklearn.cluster import DBSCAN
import numpy as np
import rerun as rr

# ===== Получение и подготовка данных LiDAR =====
lidar_data = client.getLidarData()
points = np.array(lidar_data.point_cloud, dtype=np.float32).reshape(-1, 3)

# ===== Фильтрация: исключаем "землю" (высота < 0.2 м) =====
above_ground = points[points[:, 2] > 0.2]

# ===== Кластеризация (DBSCAN) =====
clustering = DBSCAN(eps=1.0, min_samples=10).fit(above_ground[:, :2])  # кластеризация по X, Y
labels = clustering.labels_

# ===== Обработка кластеров =====
unique_labels = set(labels)
object_centers = []
object_distances = []

for label in unique_labels:
    if label == -1:
        continue  # -1 означает шум

    cluster_points = above_ground[labels == label]
    center = cluster_points.mean(axis=0)
    distance = np.linalg.norm(center[:2])
    
    object_centers.append(center)
    object_distances.append(distance)

# ===== Логгирование =====
rr.log("lidar/points", rr.Points3D(points))
rr.log("lidar/objects", rr.Points3D(np.array(object_centers)))

print(f"🔎 Обнаружено объектов по LiDAR: {len(object_centers)}")
for i, dist in enumerate(object_distances):
    print(f" - Объект {i+1}: ~{dist:.1f} м")


ValueError: Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required by DBSCAN.

In [ ]:
# ===== Построение и визуализация кругового маршрута =====

lidar_data = client.getLidarData()
points = np.array(lidar_data.point_cloud, dtype=np.float32).reshape(-1, 3)

# Визуализация
rr.log('lidar/points', rr.Points3D(points))

# Стартовая точка
start = client.getMultirotorState().kinematics_estimated.position
takeoff_alt = -15  # минус = вверх в NED
center = start
radius = 10
steps = 12  # количество точек круга

# Построение круга
trajectory = []
for i in range(steps):
    angle = 2 * pi * (i / steps)
    x = center.x_val + radius * cos(angle)
    y = center.y_val + radius * sin(angle)
    z = takeoff_alt
    trajectory.append(airgen.Vector3r(x, y, z))

# Возврат к центру
trajectory.append(airgen.Vector3r(center.x_val, center.y_val, takeoff_alt))

# Визуализация маршрута
rr.log("telemetry/path", rr.LineStrips3D([[(p.x_val, p.y_val, p.z_val) for p in trajectory]]))

In [ ]:
# ===== Плавное движение по маршруту и посадка =====

lidar_data = client.getLidarData()
points = np.array(lidar_data.point_cloud, dtype=np.float32).reshape(-1, 3)

# Визуализация
rr.log('lidar/points', rr.Points3D(points))
velocity = 3.0  # Плавная скорость

client.moveOnPathAsync(
    trajectory,
    velocity,
    300,
    airgen.DrivetrainType.ForwardOnly,
    airgen.YawMode(False, 0),
    -1, 0
).join()

# Плавная посадка
landing_point = airgen.Vector3r(center.x_val, center.y_val, -2)
client.moveToPositionAsync(landing_point.x_val, landing_point.y_val, landing_point.z_val, 1).join()

In [ ]:
# ===== Импорты и инициализация =====
from grid.model.navigation.objectinspection import ObjectInspect
from grid.model.navigation.visualservoing import VisualServoing
from grid.model.perception.detection.rt_detr import RT_DETR

import airgen
import numpy as np
import rerun as rr
import cv2
import time
from math import cos, sin, pi
from sklearn.cluster import DBSCAN

# Инициализация rerun
rr.init("bpla_path_lidar_stream", spawn=True)

# ===== Инициализация моделей и клиента =====
nav_inspect = ObjectInspect()
nav_visualservoing = VisualServoing()
detection_rt_detr = RT_DETR(use_local=True)

client = airgen.MultirotorClient()
client.confirmConnection()
client.enableApiControl(True)
client.armDisarm(True)
client.takeoffAsync().join()

# ===== Настройка параметров =====
takeoff_alt = -15  # высота полета
radius = 10
steps = 24
velocity = 2.0
obstacle_distance_thresh = 2.0  # м
trajectory = []

# ===== Получение стартовой позиции =====
state = client.getMultirotorState()
center = state.kinematics_estimated.position

# ===== Построение круговой траектории =====
for i in range(steps):
    angle = 2 * pi * (i / steps)
    x = center.x_val + radius * cos(angle)
    y = center.y_val + radius * sin(angle)
    z = takeoff_alt
    trajectory.append(airgen.Vector3r(x, y, z))

# Возврат к центру
trajectory.append(airgen.Vector3r(center.x_val, center.y_val, takeoff_alt))

# ===== Запуск перемещения по пути и потокового анализа =====
path_log = []
danger_points = []

for target in trajectory:
    # Перемещение к следующей точке
    client.moveToPositionAsync(target.x_val, target.y_val, target.z_val, velocity).join()
    
    # Получение текущей позиции
    current_pos = client.getMultirotorState().kinematics_estimated.position
    path_log.append([current_pos.x_val, current_pos.y_val, current_pos.z_val])
    
    # ===== Получение и обработка LiDAR =====
    lidar_data = client.getLidarData()
    points = np.array(lidar_data.point_cloud, dtype=np.float32).reshape(-1, 3)
    
    if points.size == 0:
        continue

    # Фильтрация по высоте (исключаем землю и траву)
    above_ground = points[points[:, 2] > 0.2]

    # Кластеризация
    clustering = DBSCAN(eps=1.0, min_samples=5).fit(above_ground[:, :2])
    labels = clustering.labels_

    object_centers = []
    for label in set(labels):
        if label == -1:
            continue
        cluster = above_ground[labels == label]
        center_point = cluster.mean(axis=0)
        dist = np.linalg.norm(center_point[:2] - np.array([current_pos.x_val, current_pos.y_val]))
        if dist < obstacle_distance_thresh:
            danger_points.append(center_point)
            print(f"🚨 Обнаружено препятствие на расстоянии ~{dist:.2f} м. Остановка.")
            break
        object_centers.append(center_point)

    # ===== Логгирование в Rerun =====
    rr.log("lidar/points", rr.Points3D(points))  # Все точки
    if len(object_centers):
        rr.log("lidar/objects", rr.Points3D(np.array(object_centers)))  # Центры объектов
    if len(danger_points):
        rr.log("lidar/danger", rr.Points3D(np.array(danger_points), colors=np.array([[255, 0, 0]]*len(danger_points), dtype=np.uint8)))  # Опасные точки
    
    rr.log("uav/position", rr.Transform3D(translation=(current_pos.x_val, current_pos.y_val, current_pos.z_val)))
    rr.log("telemetry/path", rr.LineStrips3D([path_log]))

    # Сохранение данных каждую итерацию
    np.save("lidar_points_latest.npy", points)
    np.save("trajectory_log.npy", np.array(path_log))

    # Остановка при опасности
    if len(danger_points):
        break

# ===== Завершение: плавная посадка и выключение =====
client.moveToPositionAsync(current_pos.x_val, current_pos.y_val, -2, 1).join()
client.landAsync().join()
client.armDisarm(False)

# Финальное сохранение
np.save("lidar_points_final.npy", points)
np.save("trajectory_final.npy", np.array(path_log))
print("✅ Миссия завершена. Данные сохранены.")


Error: winit EventLoopError: os error at /usr/local/cargo/registry/src/index.crates.io-6f17d22bba15001f/winit-0.30.7/src/platform_impl/linux/mod.rs:764: neither WAYLAND_DISPLAY nor WAYLAND_SOCKET nor DISPLAY is set.


[2025-05-05T10:09:45Z WARN  re_sdk_comms::buffered_client] Failed to send message after 3 attempts: Failed to connect to Rerun server at 127.0.0.1:9876: Connection refused (os error 111)


Connected!
Client Ver:1 (Min Req: 1), Server Ver:1 (Min Req: 1)



🚨 Обнаружено препятствие на расстоянии ~0.15 м. Остановка.


✅ Миссия завершена. Данные сохранены.
